In [110]:
!pip install -r requirements.txt


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [111]:
## Set up
import os
import sys
import subprocess   
import time
import pandas as pd
from dotenv import load_dotenv
import openai
import os
import numpy as np
from scipy.spatial.distance import cosine

from rank_bm25 import BM25Okapi

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

source_schema_path = './01_input/01_source_CTR.xlsx'
target_schema_path = './01_input/02_Target.xlsx'

In [112]:
#reading source and target schema files
df_source_schema = pd.read_excel(source_schema_path, header=0)
df_source_schema.columns

Index(['TRIMMED COLUMN NAME', 'Entry #', 'Subject Area',
       'Current CTR Definition Field Name', 'Data Type', 'Business Definition',
       'Usage', 'Required Field? (Y/N)', 'Notes/Values', 'CHR?', 'Table Name',
       'Column Name', 'Optum Extract Data Name', 'Optum Notes'],
      dtype='object')

In [113]:
df_source_schema.head()

,TRIMMED COLUMN NAME,Entry #,Subject Area,Current CTR Definition Field Name,Data Type,Business Definition,Usage,Required Field? (Y/N),Notes/Values,CHR?,Table Name,Column Name,Optum Extract Data Name,Optum Notes
0,ACCESS_FEE,1,Financial,CTR-ACCESS-FEE,9(7)V99,Fee to pay for access to a provider network,B,Y,NaN,N,IADS003_FINAN,ACCESS_FEE,PFIN-ACCESS-FEE,NaN
1,NNH-FEE-INDICATOR,2,Process,CTR-ACCESS-FEE-INDICATOR,NaN,Determines if the vendor was primary or second...,B,Y,NaN,NaN,IADS013_NNH,NNH-FEE-INDICATOR,PNNH-NNH-FEE-INDICATOR,NaN
2,ACCID-DT,3,Medical,CTR-ACCIDENT-DATE,"9(8), DATE",Date of accident / injury,B,Y,NaN,Y,"IADS009_MED_INST, IADS009_MED_PROF",ACCID-DT,"PINS-ACC-DT, PPROF-ACC-DT",NaN
3,ACCID-DT-TYPE,4,Medical,CTR-ACCIDENT-DATE-TYPE,X,Code for significant event occurring at ctr-ac...,B,Y,NaN,Y,"IADS009_MED_INST, IADS009_MED_PROF",ACCID-DT-TYPE,"PINS-ACC-TYPE, PPROF-ACC-TYPE",NaN
4,ACCT-ID,5,Not to be used,CTR-ACCOUNT-ID,X(4),(Not to be used) – Defines the financial check...,B,N,Concatenates the Bank ID and the Bank Seq colu...,Y,IADS102,ACCT-ID,PCRM-ACCT-ID,Defines the financial checking account


In [114]:
df_target_schema = pd.read_excel(target_schema_path, header=0)
df_target_schema.columns

Index(['OI PDI CDM Name', 'Valid Values', 'Nullable', 'Format',
       'Implementation Notes', 'Level'],
      dtype='object')

In [115]:
### Subsetting the target schema just to make sample testing faster

df_target_schema = df_target_schema.iloc[50:53]  # only from row 51 to 60 (0-based index)

In [116]:
# Helper: Create schema chunks
def make_source_schema_chunks(df, table_name):
    chunks = []
    for _, row in df.iterrows():
        # Safely get each field, defaulting to empty string if not present
        trimmed_col = str(row.get('TRIMMED COLUMN NAME', ''))
        subject_area = str(row.get('Subject Area', ''))
        data_type = str(row.get('Data Type', ''))
        business_def = str(row.get('Business Definition', ''))
        tbl_name = str(row.get('Table Name', table_name))
        col_name = str(row.get('Column Name', trimmed_col))
        opt_cols = str(row.get('Optum Extract Data Name', ''))
        
        # Compose chunk text
        chunk = f"Column name is {col_name} from the table {tbl_name} and subject area {subject_area} | Its data type is {data_type} and more information about it - {business_def}"
        if 'not to be used' in chunk.lower() or 'do not use' in chunk.lower() or 'deprecated' in chunk.lower():
            continue
        
        chunks.append({
            'trimmed_column_name': trimmed_col,
            'table_name': tbl_name,
            'column_name': col_name,
            'subject_area': subject_area,
            'data_type': data_type,
            'business_definition': business_def,
            'chunk': chunk
        })
    print(chunks)
    return chunks

def make_target_schema_chunks(df):
    chunks = []
    for _, row in df.iterrows():
        field_name = str(row.get('OI PDI CDM Name', ''))
        valid_values = str(row.get('Valid Values', ''))
        is_nullable = str(row.get('Nullable', ''))
        field_format = str(row.get('Format', ''))
        imp_notes = str(row.get('Implementation Notes', ''))
        # Compose chunk text
        chunk = f"We need to make {field_name} and its valid values are {valid_values}, is_nullable:  {is_nullable} and its format is {field_format}, special instructions: {imp_notes}"
        chunks.append({
            'field_name': field_name,
            'valid_values': valid_values,
            'is_nullable': is_nullable,
            'field_format': field_format,
            'imp_notes': imp_notes,
            'chunk': chunk
        })
    print(chunks)
    return chunks

In [117]:
# Create source and target schema chunks
source_chunks = make_source_schema_chunks(df_source_schema, "source_table")

[{'trimmed_column_name': 'ACCESS_FEE', 'table_name': 'IADS003_FINAN', 'column_name': 'ACCESS_FEE', 'subject_area': 'Financial', 'data_type': '9(7)V99', 'business_definition': 'Fee to pay for access to a provider network', 'chunk': 'Column name is ACCESS_FEE from the table IADS003_FINAN and subject area Financial | Its data type is 9(7)V99 and more information about it - Fee to pay for access to a provider network'}, {'trimmed_column_name': 'NNH-FEE-INDICATOR', 'table_name': 'IADS013_NNH', 'column_name': 'NNH-FEE-INDICATOR', 'subject_area': 'Process', 'data_type': 'nan', 'business_definition': 'Determines if the vendor was primary or secondary network', 'chunk': 'Column name is NNH-FEE-INDICATOR from the table IADS013_NNH and subject area Process | Its data type is nan and more information about it - Determines if the vendor was primary or secondary network'}, {'trimmed_column_name': 'ACCID-DT', 'table_name': 'IADS009_MED_INST, IADS009_MED_PROF', 'column_name': 'ACCID-DT', 'subject_area

In [118]:
target_chunks = make_target_schema_chunks(df_target_schema)


[{'field_name': 'Claim Total Billed Amount', 'valid_values': 'nan', 'is_nullable': 'Yes', 'field_format': 'nan', 'imp_notes': 'nan', 'chunk': 'We need to make Claim Total Billed Amount and its valid values are nan, is_nullable:  Yes and its format is nan, special instructions: nan'}, {'field_name': 'Claim Total Patient Liability Amount', 'valid_values': 'nan', 'is_nullable': 'Yes', 'field_format': 'nan', 'imp_notes': 'nan', 'chunk': 'We need to make Claim Total Patient Liability Amount and its valid values are nan, is_nullable:  Yes and its format is nan, special instructions: nan'}, {'field_name': 'Claim Total Patient CoPay Amount', 'valid_values': 'nan', 'is_nullable': 'Yes', 'field_format': 'nan', 'imp_notes': 'nan', 'chunk': 'We need to make Claim Total Patient CoPay Amount and its valid values are nan, is_nullable:  Yes and its format is nan, special instructions: nan'}]


In [119]:
# Helper: Get embeddings for a list of chunks
def get_embeddings(chunks):
    texts = [c['chunk'] for c in chunks]
    response = openai.embeddings.create(input=texts, model="text-embedding-ada-002")
    for i, emb in enumerate(response.data):
        chunks[i]['embedding'] = emb.embedding
    return chunks

source_chunks = get_embeddings(source_chunks)
target_chunks = get_embeddings(target_chunks)



In [120]:
from rank_bm25 import BM25Okapi

# Helper: Tokenize for BM25
def tokenize(text):
    return text.lower().split()

# Prepare BM25 corpus for source chunks
bm25_corpus = [tokenize(chunk['chunk']) for chunk in source_chunks]
bm25 = BM25Okapi(bm25_corpus)

# Helper: Find top N relevant source fields for a target chunk using both semantic and BM25
def find_top_n_sources_combined(target_chunk, source_chunks, n=10, bm25_weight=0.5, emb_weight=0.5):
    # Semantic similarity (cosine)
    target_emb = target_chunk['embedding']
    semantic_scores = []
    for s in source_chunks:
        sim = 1 - cosine(target_emb, s['embedding'])
        semantic_scores.append(sim)
    # BM25 similarity
    query_tokens = tokenize(target_chunk['chunk'])
    bm25_scores = bm25.get_scores(query_tokens)
    # Combine scores
    combined_scores = []
    for i in range(len(source_chunks)):
        score = bm25_weight * bm25_scores[i] + emb_weight * semantic_scores[i]
        combined_scores.append((source_chunks[i], score))
    combined_scores.sort(key=lambda x: x[1], reverse=True)
    return [s[0] for s in combined_scores[:n]]


In [121]:


# Example usage in your main loop:
all_rules = []
for target_chunk in target_chunks:
    relevant_sources = find_top_n_sources_combined(target_chunk, source_chunks, n=5)
    print(f"\nTarget Chunk: {target_chunk['chunk']}")
    print("Top 20 Relevant Source Chunks (BM25 + Semantic):")
    for idx, s in enumerate(relevant_sources, 1):
        print(f"{idx}. {s['chunk']}")
    # Prepare JSON for LLM
    relevant_source_json = [
        {
            #'trimmed_column_name': s.get('trimmed_column_name', ''),
            #'table_name': s.get('table_name', ''),
            #'column_name': s.get('column_name', ''),
            #'subject_area': s.get('subject_area', ''),
            #'data_type': s.get('data_type', ''),
            #'business_definition': s.get('business_definition', ''),
            'summary': s.get('chunk', '')
        }
        for s in relevant_sources
    ]
    target_json = {
        #'field_name': target_chunk.get('field_name', ''),
        #'valid_values': target_chunk.get('valid_values', ''),
        #'is_nullable': target_chunk.get('is_nullable', ''),
        #'field_format': target_chunk.get('field_format', ''),
        #'imp_notes': target_chunk.get('imp_notes', ''),
        'summary': target_chunk.get('chunk', '')
    }
    prompt = (
        f"Given the following relevant source fields (as JSON):\n{relevant_source_json}\n\n"
        f"And the following target field (as JSON):\n{target_json}\n\n"
        "nan or na shows that the field is not specified, hence you can ignore that field in your reasoning. \n"
        "Special focus on interpreting the field name and valid values to understand the target field requirements.\n"
        "Provide a transformation rule in this JSON format:\n"
        "{\n"
        "  \"target_table\": \"<target_table_name>\",\n"
        "  \"target_field\": \"<target_field_name>\",\n"
        "  \"source_table\": \"<source_table_name or list>\",\n"
        "  \"source_field\": \"<source_field_name or list>\",\n"
        "  \"transformation\": \"<describe transformation or mapping logic>\"\n"
        "}\n"
        "Be explicit about any type conversions, calculations, or renaming."
    )
    print("\nPrompt sent to LLM:\n", prompt)
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a data engineer who writes data transformation rules."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=500,
        temperature=0.2
    )
    all_rules.append(response.choices[0].message.content)



Target Chunk: We need to make Claim Total Billed Amount and its valid values are nan, is_nullable:  Yes and its format is nan, special instructions: nan
Top 20 Relevant Source Chunks (BM25 + Semantic):
1. Column name is ADJST-AMT from the table IADS006_PROCS and subject area Financial | Its data type is S9(7)V99 and more information about it - Total adjustment amount applied for a claim
2. Column name is CHECK-AMT from the table IADS006_PROCS and subject area Process | Its data type is S9(7)V99 and more information about it - Total amount of the check this claim was included on
3. Column name is LT-MED-ADJ-AMT from the table IADS003_FINAN and subject area Financial | Its data type is S9(9)V99 and more information about it - Total amount of lifetime medical benefits paid to date
4. Column name is GROSS-PAY from the table IADS004_REV and subject area Revenue | Its data type is S9(7)V99 and more information about it - Negotiated amount the provider has agreed to accept as payment in full

In [122]:
all_rules

['```json\n{\n  "target_table": "Claim_Data",\n  "target_field": "Claim_Total_Billed_Amount",\n  "source_table": ["IADS006_PROCS", "IADS003_FINAN", "IADS004_REV", "IADS001_BILL_LINE"],\n  "source_field": ["ADJST-AMT", "LT-MED-ADJ-AMT", "GROSS-PAY", "OI_PAID_AMT"],\n  "transformation": "Calculate the sum of ADJST-AMT, LT-MED-ADJ-AMT, GROSS-PAY, and OI_PAID_AMT to derive the total billed amount for the claim."\n}\n```',
 '{\n  "target_table": "Claim",\n  "target_field": "Total_Patient_Liability_Amount",\n  "source_table": ["IADS001_BILL_LINE", "IADS006_PROCS"],\n  "source_field": ["PAT_LIAB", "ADJST-AMT"],\n  "transformation": "Calculate the sum of PAT_LIAB from table IADS001_BILL_LINE and ADJST-AMT from table IADS006_PROCS to derive the total patient liability amount for a claim."\n}',
 '```json\n{\n  "target_table": "Claim",\n  "target_field": "TotalPatientCoPayAmount",\n  "source_table": ["IADS001_BILL_LINE", "IADS006_PROCS"],\n  "source_field": ["PAT_LIAB", "ADJST-AMT"],\n  "transfor

In [123]:

# Save all rules to 03_output
os.makedirs('./03_output', exist_ok=True)
with open('./03_output/transformation_rules.json', 'w') as f:
    for rule in all_rules:
        f.write(rule + "\n")
